In [1]:
import os
import numpy as np
import pandas as pd
from get_params import DATA_FOLDER, SHARED_DATA_FOLDER
from get_params import INPUT_DATASET_EVAL_FPATH, INPUT_DATASET_DPO_LABELS_FPATH

## What data to run on
EVAL_FPATH = INPUT_DATASET_EVAL_FPATH
EVAL_OUT_FPATH = INPUT_DATASET_DPO_LABELS_FPATH

In [2]:
eval_df = pd.read_csv(EVAL_FPATH)


In [3]:
eval_df.head(2)

,Unnamed: 0,title,body,topic,standard_prompt,empathy_prompt,standard_output,empathy_output,id,winner_adoptability,winner_empathy,winner_clarity,winner_comprehensiveness,winner_practical,winner_overall,justification
0,0,Feeling Overwhelmed with Potty Training,My 1-year-old is showing interest in the potty...,Potty training struggles - Ages 1-2,<s>[INST] You are a parenting assistant. Your ...,<s>[INST] You are a parenting assistant. Your ...,It's completely normal to feel overwhelmed dur...,"I understand your feelings of overwhelm, and i...",1,System A,System A,System A,System A,System B,System A,"System A's response is more comprehensive, cov..."
1,1,Encouraging Progress but Unsure,"Yesterday, my 1-year-old actually sat on the p...",Potty training struggles - Ages 1-2,<s>[INST] You are a parenting assistant. Your ...,<s>[INST] You are a parenting assistant. Your ...,Congratulations on your child's progress! It's...,Congratulations on your child's progress! It's...,2,System A,Tie,System A,System A,System A,System A,System A provides a more comprehensive and str...


In [4]:
## Assigning chosen vs rejected responses
def choose_a(row):
    row["chosen_system"] = "system a"
    row["chosen"] = row["standard_output"]
    row["rejected"] = row["empathy_output"]
    return row
    
def choose_b(row):
    row["chosen_system"] = "system b"
    row["chosen"] = row["empathy_output"]
    row["rejected"] = row["standard_output"]
    return row

## Selecting chosen vs rejected response
Simply choose overall winner as chosen


In [6]:
def get_chosen_rejected(row):
    if row["winner_overall"].lower() == "system a":
        row = choose_a(row)
    elif row["winner_overall"].lower() == "system b":
        row = choose_b(row)
    else:
        raise Exception("winner_overall should never be a tie.")
    return row

eval_df_out = eval_df.apply(get_chosen_rejected, axis=1)


## Print some stats about the selected dataset
cols = [
    "id",
    "title",
    "body",
    "topic",
    "chosen",
    "rejected"
]
print (len(eval_df_out))
display(eval_df_out[cols].head(2))

## Output to file
eval_df_out[cols].to_json(EVAL_OUT_FPATH, orient="records", lines=True)


1182


,id,title,body,topic,chosen,rejected
0,1,Feeling Overwhelmed with Potty Training,My 1-year-old is showing interest in the potty...,Potty training struggles - Ages 1-2,It's completely normal to feel overwhelmed dur...,"I understand your feelings of overwhelm, and i..."
1,2,Encouraging Progress but Unsure,"Yesterday, my 1-year-old actually sat on the p...",Potty training struggles - Ages 1-2,Congratulations on your child's progress! It's...,Congratulations on your child's progress! It's...


## [Old V1] EMPATHY FOCUSSED - Selecting chosen vs rejected response

#### Filter/ Excluded samples:
- **Remove any tied on empathy**: Since our goal is to tune mistral to be more empathetic while preserve quality, we exclude any samples that are tied on empathy. 
- **Overall not a tie, but empathy mismatch:** If the overall winner is System X but the empathy winner is System Y, label the sample mismatched and discard it. 

#### Choosing a response for DPO
- **Consistent Win:** If the overall winner is System X and the empathy winner is System X, select System X’s response.
- **Tie overall, empathy to X:** If the overall winner is a tie and the empathy winner is System X, select System X’s response.

**Net effect:** the final chosen response is always the better (or equally good) and the more empathetic (or equally empathetic) option.

In [ ]:
## Remove any tied on empathy
eval_df_out = eval_df[(eval_df["winner_empathy_tone"].str.lower() != "tie")]
print (f"After removing empathy ties = {(len(eval_df_out))} out of {len(eval_df)} samples remain")

## Overall not a tie, but empathy mismatch
eval_df_out = eval_df_out[~((eval_df_out["winner_overall"].str.lower() != "tie") & 
                           (eval_df_out["winner_overall"].str.lower() != eval_df_out["winner_empathy_tone"].str.lower())
                          )] 
print (f"After removing overall mismatch with empathy = {(len(eval_df_out))} out of {len(eval_df)} samples remain")

def get_chosen_rejected(row):
    ## Consistent winner is A
    if row["winner_overall"].lower() == "system a": # and we've already excluded samples with tied empathy
        row = choose_a(row)
    ## Consistent winner is B
    elif row["winner_overall"].lower() == "system b":
        row = choose_b(row)
    ## Tie overall and empathy winner is A
    elif row["winner_empathy_tone"].lower() == "system a":
        row = choose_a(row)
    ## Tie overall and empathy winner is B
    elif row["winner_empathy_tone"].lower() == "system b":
        row = choose_b(row)
    else:
        raise Exception("winner_empathy_tone should never be a tie.")
    return row

eval_df_out = eval_df_out.apply(get_chosen_rejected, axis=1)


## Print some stats about the selected dataset
cols = [
    "id",
    "title",
    "body",
    "topic",
    "chosen",
    "rejected",
    "chosen_system",
    "winner_overall",
    "winner_empathy_tone"
]
print (len(eval_df_out))
display(eval_df_out[cols].head(2))
display(eval_df_out["chosen_system"].value_counts())
display(eval_df_out["chosen_system"].value_counts(normalize=True))
display(eval_df_out["winner_empathy_tone"].value_counts())
display(eval_df_out["winner_empathy_tone"].value_counts(normalize=True))

## Output to file - This is V1 so commenting out
# eval_df_out[cols].to_json(EVAL_OUT_FPATH, orient="records", lines=True)

